<a href="https://colab.research.google.com/github/linhlinhle997/poem-generation-gpt2/blob/develop/poem_generation_gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 117.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import os
import math
import pandas as pd

import torch
from transformers import (
    GPT2Tokenizer,
    GPT2LMHeadModel,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer
)
from huggingface_hub import notebook_login
from datasets import Dataset

## Prepare the dataset

In [5]:
DATASET_PATH = "poem_data.csv"

df = pd.read_csv(DATASET_PATH)
df.head()

,title,content,source,url
0,Bạn xấu như chiếc bóng,Bạn xấu như chiếc bóng\nCứ bám riết theo anh\n...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,Cái làm ta hạnh phúc,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,Chiều vừa xốp trên tay,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
3,Chơi thân không có nghĩa,Chơi thân không có nghĩa\nKhông cãi nhau bao g...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,Có thể buồn chút ít,"Có thể buồn chút ít\nMột mình, không người yêu...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...


In [6]:
df["content"][0]

'Bạn xấu như chiếc bóng\nCứ bám riết theo anh\nKhi anh sáng, rực rỡ\nNhư mặt trời long lanh\n\nNhưng họ sẽ biến mất\nKhi trời phủ mây đen\nTức là khi anh đói\nTrong túi không có tiền'

Splits the given content into four-line poem segments.

In [7]:
def split_content(content):
    samples = []

    # Split content into separate stanzas based on \n\n
    poem_parts = content.split("\n\n")
    for poem_part in poem_parts:
        # Split each stanza into individual lines
        poem_lines = poem_part.split("\n")

        # Keep only stanzas that have exactly 4 lines
        if len(poem_lines) == 4:
            samples.append(poem_lines)
    return samples


df["content"] = df["content"].apply(split_content)
df.head()

,title,content,source,url
0,Bạn xấu như chiếc bóng,"[[Bạn xấu như chiếc bóng, Cứ bám riết theo anh...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,Cái làm ta hạnh phúc,"[[Cái làm ta hạnh phúc, Thực ra cũng chẳng nhi...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,Chiều vừa xốp trên tay,"[[Chiều vừa xốp trên tay, Chợt nghe thoáng ong...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
3,Chơi thân không có nghĩa,"[[Chơi thân không có nghĩa, Không cãi nhau bao...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,Có thể buồn chút ít,"[[Có thể buồn chút ít, Một mình, không người y...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...


Convert the "content" column containing lists into multiple separate rows.




In [8]:
df_exploded = df.explode("content")
df_exploded.reset_index(drop=True, inplace=True)
df_exploded = df_exploded.dropna(subset=["content"])
df_exploded.head()

,title,content,source,url
0,Bạn xấu như chiếc bóng,"[Bạn xấu như chiếc bóng, Cứ bám riết theo anh,...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,Bạn xấu như chiếc bóng,"[Nhưng họ sẽ biến mất, Khi trời phủ mây đen, T...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,Cái làm ta hạnh phúc,"[Cái làm ta hạnh phúc, Thực ra cũng chẳng nhiề...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
3,Cái làm ta hạnh phúc,"[Rồi thêm chút công việc, Cho ta làm hàng ngày...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,Chiều vừa xốp trên tay,"[Chiều vừa xốp trên tay, Chợt nghe thoáng ong ...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...


Converts lists into multiline strings by joining list elements with \n, making the text more readable.

In [9]:
df_exploded["content"] = df_exploded["content"].apply(lambda x: "\n".join(x))
df_exploded

,title,content,source,url
0,Bạn xấu như chiếc bóng,Bạn xấu như chiếc bóng\nCứ bám riết theo anh\n...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,Bạn xấu như chiếc bóng,Nhưng họ sẽ biến mất\nKhi trời phủ mây đen\nTứ...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,Cái làm ta hạnh phúc,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
3,Cái làm ta hạnh phúc,Rồi thêm chút công việc\nCho ta làm hàng ngày\...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,Chiều vừa xốp trên tay,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
...,...,...,...,...
216,Ám ảnh sông xưa,"Ôi, con sóng chết khô,\nvật vờ trong bùn quánh...",NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...
217,Áng dương không biết sầu,Áng dương không biết sầu \nNằm mãi ở trên cao\...,"Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...
218,Áng dương không biết sầu,Em ơi sao tàn nhẫn \nNỡ xa rời vòng tay \nMộ...,"Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...
219,Áng dương không biết sầu,Đợi em những đêm dài \nKhông một lời đối đáp\n...,"Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...


## Data Preprocessing

onverts a Pandas DataFrame into a Hugging Face Dataset

In [10]:
poem_dataset = Dataset.from_pandas(df_exploded)
poem_dataset

Dataset({
    features: ['title', 'content', 'source', 'url', '__index_level_0__'],
    num_rows: 200
})

Splits the dataset into training and test sets

In [11]:
TEST_SIZE = 0.1
poem_dataset = poem_dataset.train_test_split(test_size=TEST_SIZE)
poem_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'content', 'source', 'url', '__index_level_0__'],
        num_rows: 180
    })
    test: Dataset({
        features: ['title', 'content', 'source', 'url', '__index_level_0__'],
        num_rows: 20
    })
})

In [12]:
MODEL_NAME = "danghuy1999/gpt2-viwiki"

tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/773k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/431k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/916 [00:00<?, ?B/s]

In [16]:
MAX_SEQ_LEN = 100
tokenizer.pad_token = tokenizer.eos_token # Use the <EOS> token as the padding token.

def preprocess_fnc(row):
    return tokenizer(
        row["content"],
        max_length=MAX_SEQ_LEN,
        padding="max_length",
        truncation=True
    )

tokenizer_poem_dataset = poem_dataset.map(
    preprocess_fnc,
    batched=True,
    num_proc=4,
    remove_columns=poem_dataset["train"].column_names
)

Map (num_proc=4):   0%|          | 0/180 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/20 [00:00<?, ? examples/s]

Creates a data collator that batches tokenized data and dynamically handles padding.
- Automatically pads sequences to the longest in the batch.
- mlm=False -> Prepares data for causal language modeling (CLM - GPT). (mlm=True -> MLM - BERT)


In [17]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer,
    mlm=False
)

## Train Model

In [18]:
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [21]:
training_args = TrainingArguments(
    output_dir="./gpt2_poem_generation",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=10,
    weight_decay=0.01,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenizer_poem_dataset["train"],
    eval_dataset=tokenizer_poem_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

<ipython-input-21-6f7eb0f24a4d>:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=230, training_loss=5.990230659816576, metrics={'train_runtime': 177.7132, 'train_samples_per_second': 10.129, 'train_steps_per_second': 1.294, 'total_flos': 91860480000000.0, 'train_loss': 5.990230659816576, 'epoch': 10.0})

## Inference

In [41]:
device = "cuda" if torch.cuda.is_available() else "cpu"

prompt = "Ly cà phê buổi sáng\n"
inputs = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

outputs = model.generate(
    inputs,
    max_length=50, # Limits the generated text to 50 tokens.
    do_sample=True, # Enables sampling, instead of always picking the most probable token
    top_k=50, # Considers only the top 50 most probable tokens for selection.
    top_p=0.95, # Uses nucleus sampling, selecting tokens until their cumulative probability reaches 95%.
    temperature=0.8, # Controls randomness (lower values make output more deterministic).
    repetition_penalty=1.2, # Controls randomness (lower values make output more deterministic).
    pad_token_id=tokenizer.eos_token_id
)

results = tokenizer.batch_decode(outputs, skip_special_tokens=True)
results = results[0]

for line in results.split("\n"):
    print(line)

Ly cà phê buổi sáng
Người bạn đang cầu vui
Một người chơi cho chiếc áo
Những đoạn đầu sau, ngay cả cha mẹ
Nhưng không còn ai gặp gỡ? Mặc dù vậy
Hai điều lệ này để tạo nên một chiến thắng
